In [1]:
from datetime import datetime, timedelta
import vectorbt as vbt
from get_data import select_data
import talib
from pprint import pprint
import numpy as np
import pandas as pd
from itertools import combinations, product
from tools.dataframe import apply_logical_operation
import json
from typing import Optional, Union, List

vbt.settings.portfolio["fees"] = 0.001
vbt.settings.portfolio["slippage"] = 0.0025

from_date = datetime.now() - timedelta(days=2 * 365)
end_date = datetime.now()
interval = "30m"
symbol = ["BTCUSDT", "ETHUSDT"]
price = select_data(symbol, interval, from_date, end_date)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [65]:
def apply_trix(close, trix_window, signal_window):
    close_df = pd.DataFrame(close)
    close_df = close_df.apply(talib.TRIX, axis=0, timeperiod=trix_window)
    signal_df = close_df.apply(talib.SMA, axis=0, timeperiod=signal_window)
    return close_df, signal_df


def plot_trix(trix, signal, column=None, fig=None):
    fig = trix.vbt.plot(fig=fig)
    fig = signal.vbt.plot(fig=fig)


TrixIndicator = vbt.IndicatorFactory(
    input_names=["close"],
    param_names=["trix_window", "signal_window"],
    output_names=["trix", "signal"],
    subplots=dict(
        plot_outputs=dict(
            plot_func=plot_trix,
            resolve_trix=True,
            resolve_signal=True,
        )
    ),
).from_apply_func(
    apply_trix,
    trix_window=5,
    signal_window=10,
)


def trix_strategy(
    price,
    rr_ratio=0.5,
    sl_stop=0.15,
    sl_trail=False,
    stoch_rsi_k_window=300,
    trix_trix_window=500,
    trix_signal_window=2000,
    ma_window=3500,
    percent_k_below=80,
    percent_k_above=20,
    validation=False,
    **kwargs,
):
    if isinstance(price, tuple) or isinstance(price, List) or isinstance(price, dict):
        try:
            if isinstance(price, dict):
                price = select_data(**price)
            else:
                price = select_data(*price)
        except Exception:
            raise ValueError(
                "price list must be like (symbol, interval, from_date, end_date)"
            )

    if validation:
        stoch_rsi_k_window = np.linspace(50, 1000, 5, dtype=int)
        n = 5
        trix_trix_window, trix_signal_window = vbt.utils.params.create_param_combs(
            (
                product,
                np.linspace(2, 2000, n, dtype=int),
                np.linspace(2, 2000, n, dtype=int),
            )
        )

    stoch_rsi = vbt.STOCH.run(
        high=price.get("High"),
        low=price.get("Low"),
        close=price.get("Close"),
        k_window=stoch_rsi_k_window,
        run_unique=True,
        short_name="stoch_rsi",
    )

    trix = TrixIndicator.run(
        price.get("Close"),
        trix_window=trix_trix_window,
        signal_window=trix_signal_window,
        run_unique=True,
        short_name="trix",
    )
    trend_ma = vbt.MA.run(
        price.get("Close"), window=ma_window, ewm=True, run_unique=True
    )

    trend_signals = trend_ma.ma_below(price.get("Close"))
    trix_signals = trix.trix_above(trix.signal)
    exit_signals = trix.trix_below(trix.signal)
    stoch_rsi_entry_signals = stoch_rsi.percent_k_below(percent_k_below)
    stoch_rsi_exit_signals = stoch_rsi.percent_k_above(percent_k_above)

    entries = apply_logical_operation(
        trix_signals & trend_signals, stoch_rsi_entry_signals, "&"
    )
    exits = apply_logical_operation(exit_signals, stoch_rsi_exit_signals, "&")

    ohlcstcx = vbt.OHLCSTCX.run(
        entries=entries,
        open=price.get("Open"),
        high=price.get("High"),
        low=price.get("Low"),
        close=price.get("Close"),
        sl_stop=sl_stop,
        sl_trail=sl_trail,
        tp_stop=sl_stop / rr_ratio,
    )

    entries = ohlcstcx.new_entries
    exits = apply_logical_operation(ohlcstcx.exits, exits, "|")

    return entries, exits

In [105]:
entries, exits = trix_strategy(
    (symbol, interval, from_date, end_date), sl_stop=0.1, validation=True
)
pf = vbt.Portfolio.from_signals(
    price.get("Close"), entries=entries, exits=exits, freq=timedelta(minutes=30)
)
total_return = pf.total_return()
pprint(
    {
        name: value
        for name, value in zip(total_return.index.names, total_return.idxmax())
    }
)

{'ma_ewm': True,
 'ma_window': 3500,
 'ohlcstcx_sl_stop': 0.1,
 'ohlcstcx_sl_trail': False,
 'ohlcstcx_tp_stop': 0.2,
 'stoch_rsi_k_window': 50,
 'symbol': 'ETHUSDT',
 'trix_signal_window': 1001,
 'trix_trix_window': 501}


In [132]:
def allbut(*names, levels=[]):
    names = set(names)
    return [item for item in levels if item not in names]

mean_total_return = (
    total_return.groupby(level=allbut("symbol", levels=total_return.index.names))
    .mean()
    .sort_values(ascending=False)
)

mean_max_drawdowns = (
    pf.max_drawdown().groupby(level=allbut("symbol", levels=pf.max_drawdown().index.names))
    .mean()
    .sort_values(ascending=False)
)

index_max_total_return = mean_total_return.index[0:20]
index_min_drawdown = mean_max_drawdowns.index[0:20]
intersection_index = index_max_total_return.intersection(index_min_drawdown)

print(intersection_index)

for index in intersection_index:
    print(total_return.loc[index])
    print(mean_max_drawdowns.loc[index])
    print()

def convert_multiindex_to_dict(multiiindex, index_to_convert):
    if isinstance(index_to_convert, int):
        index = multiiindex[index_to_convert]
    else:
        index = index_to_convert
    return {
        index_name: index_value
        for index_name, index_value in zip(multiiindex.names, index)
    }


# best_params = convert_multiindex_to_dict(intersection_index, 1)
pprint(best_params)

MultiIndex([(0.1, False, 0.2, 3500, True, 1001, 2, 525)],
           names=['ohlcstcx_sl_stop', 'ohlcstcx_sl_trail', 'ohlcstcx_tp_stop', 'ma_window', 'ma_ewm', 'trix_trix_window', 'trix_signal_window', 'stoch_rsi_k_window'])


/tmp/ipykernel_3876/652424591.py:24: PerformanceWarning:

indexing past lexsort depth may impact performance.



KeyError: 1001

In [120]:
price = select_data(["BTCUSDT", "ETHUSDT", "LTCUSDT", "ADAUSDT", "DOTUSDT", "DOGEUSDT"], interval, from_date, end_date)
entries, exits = trix_strategy(
    price, sl_stop=0.1, **best_params
)
pf = vbt.Portfolio.from_signals(
    price.get("Close"), entries=entries, exits=exits, freq=timedelta(minutes=30)
)
total_return = pf.total_return()

In [122]:
pf.iloc[0].returns_stats()
pf.iloc[1].returns_stats()
pf.iloc[2].returns_stats()
pf.iloc[3].returns_stats()
pf.iloc[4].returns_stats()
pf.iloc[5].returns_stats()

Start                        2020-01-17 00:00:00+00:00
End                          2022-01-15 23:30:00+00:00
Period                               728 days 12:30:00
Total Return [%]                            147.625487
Benchmark Return [%]                       7802.852737
Annualized Return [%]                        57.506125
Annualized Volatility [%]                   117.809366
Max Drawdown [%]                             81.451171
Max Drawdown Duration                273 days 19:30:00
Sharpe Ratio                                  0.973678
Calmar Ratio                                   0.70602
Omega Ratio                                    1.05601
Sortino Ratio                                 1.440818
Skew                                          1.149691
Kurtosis                                     130.68919
Tail Ratio                                    1.021737
Common Sense Ratio                            1.609298
Value at Risk                                -0.006762
Alpha     

In [133]:
total_return

# pf.iloc[0].returns_stats()
# pf.iloc[1].returns_stats()

#     y_level="custom_signal_window",
#     z_level="stoch_k_window",
#     slider_level="symbol",
#     trace_kwargs=dict(colorbar=dict(title="Total return", tickformat="%")),
# )
# fig.show()

ohlcstcx_sl_stop  ohlcstcx_sl_trail  ohlcstcx_tp_stop  ma_window  ma_ewm  symbol    trix_trix_window  trix_signal_window  stoch_rsi_k_window
0.1               False              0.2               3500       True    BTCUSDT   1001              2                   525                   3.255619
                                                                          ETHUSDT   1001              2                   525                   3.500419
                                                                          LTCUSDT   1001              2                   525                   0.317541
                                                                          ADAUSDT   1001              2                   525                   0.350199
                                                                          DOTUSDT   1001              2                   525                   1.438726
                                                                          DOGEUSDT  1001      